In [ ]:
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip install seaborn
!pip install lightgbm
!pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from lightgbm import LGBMRegressor


In [4]:
test =  pd.read_csv('../data/raw/bike_sharing_test.csv') #hago el eda en este
df_raw = pd.read_csv('../data/raw/bike_sharing_train.csv')


In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [7]:
df_raw.sample(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
4543,2011-11-02 09:00:00,4,0,1,1,13.94,18.180,87,0.0000,15,213,228
2073,2011-05-13 07:00:00,2,0,1,3,20.50,24.240,88,11.0014,16,141,157
829,2011-02-18 00:00:00,1,0,1,1,18.04,21.970,72,19.0012,12,20,32
9778,2012-10-11 19:00:00,4,0,1,1,18.04,21.970,51,8.9981,81,662,743
7116,2012-04-14 21:00:00,2,0,0,2,22.96,26.515,43,8.9981,53,167,220
7745,2012-06-03 02:00:00,2,0,0,1,21.32,25.000,59,7.0015,22,67,89
64,2011-01-03 19:00:00,1,0,1,1,8.20,12.880,47,0.0000,8,102,110
2410,2011-06-08 08:00:00,2,0,1,1,31.16,35.605,58,11.0014,46,435,481
5451,2012-01-02 06:00:00,1,1,0,1,10.66,11.365,41,26.0027,0,14,14
4998,2011-12-02 08:00:00,4,0,1,1,9.84,14.395,87,0.0000,13,432,445
